In [1]:
import psycopg2
import os
import pandas as pd
import numpy as np

In [ ]:
params = {
    "host": os.environ['APP_POSTGRES_HOST'],
    "port": os.environ['APP_POSTGRES_PORT'],
    "user": 'postgres'
}
conn = psycopg2.connect(**params)
psycopg2.extensions.register_type(
    psycopg2.extensions.UNICODE,
    conn
)
conn.set_isolation_level(
    psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT
)
cursor = conn.cursor()

In [ ]:
tickets="select * from tickets"
users="select * from users"
chat="select * from chat"
chat_messages="select * from chat_messages"
notices="select * from notices"
hard_tickets="""CREATE VIEW places AS SELECT resolved_by_id as expert_id, count(id) as count 
                    FROM tickets WHERE resolved_by_id IS NOT NULL GROUP BY resolved_by_id 
                        ORDER BY count DESC
                CREATE VIEW ranks AS SELECT name, expert_id, count, row_number() over (order by count asc) as place 
                    FROM places JOIN users ON places.expert_id=users.id
                with t1 as (SELECT * FROM notices) SELECT ticket_id as ticket, count(type) as count_tag, array_agg(type) as types, array_agg(text) as notices, array_agg(user_id) as experts, round(avg(place),1) as difficulty 
                    FROM t1 JOIN ranks ON user_id=expert_id WHERE  type <> 'SOLVED' GROUP BY ticket 
                        ORDER BY count_tag DESC, difficulty DESC LIMIT 20
"""

#создаем объект tickets
cursor.execute(tickets)
tickets=pd.DataFrame([a for a in cursor.fetchall()], columns=['id', 'has_attachment', 'status', 'text', 'user_id', 'resolved_by', 'target_id', 'visits_count_admin', 'visits_count_expert', 'visits_count_user'])
conn.commit()
tickets.head()

#создаем объект users
cursor.execute(users)
users=pd.DataFrame([a for a in cursor.fetchall()], columns=['id', 'email', 'login', 'mailing_enable', 'name', 'status', 'type', 'oauth_source'])
conn.commit()
users.head()

#создаем объект chat
cursor.execute(chat)
chat=pd.DataFrame([a for a in cursor.fetchall()], columns=['id', 'target_id', 'ticket_id'])
conn.commit()
chat.head()

#создаем объект chat_messages
cursor.execute(chat_messages)
chat_messages=pd.DataFrame([a for a in cursor.fetchall()], columns=['id', 'text', 'author_id', 'chat_id'])
conn.commit()
chat_messages.head()

#создаем объект notices
cursor.execute(notices)
notices=pd.DataFrame([a for a in cursor.fetchall()], columns=['id', 'ticket_id', 'related_type', 'text', 'type', 'user_id'])
conn.commit()
notices.head()

#создаем объект hard_tickets
cursor.execute(hard_tickets)
hard_tickets=pd.DataFrame([a for a in cursor.fetchall()], columns=['ticket_id', 'count_tag', 'types', 'notices', 'experts', 'difficulty'])
conn.commit()
hard_tickets.head()

In [ ]:
# Получение списка чатов с наибольшим количеством сообщений. Задача - найти инсайты для 
# расширения возможностей сервиса элементами соцсети/мессенджера 
# в первую очередь для возможной мобильной версии сервиса.
# select chat_id, count(chat_id) as count FROM chat_messages GROUP BY chat_id ORDER BY count DESC LIMIT 10;

df_chats=chat_messages.groupby('chat_id').count().sort_values(ascending=false)
df_chats.to_csv('data.csv', encoding='utf-8')
df_chats

In [ ]:
# Получение распределения авторизаций в сервисе через профили соцсетей уникальными пользователям:
# SELECT oauth_source, count(oauth_source) FROM users WHERE oauth_source IS NOT NULL GROUP BY oauth_source;

df_openid=users.groupby[pd.notnull(users.oauth_source)].count()

In [ ]:
#Определим количество заметок в каждом типе:
df_notices=notices.groupby('type').count().sort_values(ascending=False)

In [ ]:
#Определим число тикетов с атачем и без:
df_tickets=tickets.groupby('has_attachment').count()

In [ ]:
#Определим сколько тикетов находятся в том или ином статусе:
df_status=tickets.groupby('resolved_by_id').count()